### Imports & Env Setup

In [3]:
%reload_ext autoreload
%autoreload 2
import sys
import os
from dotenv import load_dotenv
load_dotenv()
from datasets import load_dataset


import dspy
sys.path.append(os.path.abspath('../'))
from benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro, llama_mmlu

### Configuration

In [7]:
NUM_THREADS = 48

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1' , # or api_base ?
    api_key = "dummy",
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1', # or api_base ?
    api_key = "dummy",

    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = llama_mmlu

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("You are a helpful assistant.") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    max_errors = 500,
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
    provide_traceback=True
)

### Load dataset

In [8]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.1,
)

len(trainset), len(valset), len(testset)

(1403, 1263, 11369)

### Baseline Benchmark

In [9]:
%%time
print("BASE PROMPT:\n", program.signature.instructions)

BASE PROMPT:
 You are a helpful assistant.
CPU times: user 270 μs, sys: 7 μs, total: 277 μs
Wall time: 231 μs


In [10]:
%%time
print("BEST EXAMPLES:\n", program.demos)

BEST EXAMPLES:
 []
CPU times: user 107 μs, sys: 0 ns, total: 107 μs
Wall time: 110 μs


In [11]:
%%time

print("Starting execution...")
# eval_subset_size = len(testset)
score, results, all_scores = evaluate(
    program,
    devset=testset[:40],
)

Starting execution...
Average Metric: 35.00 / 40 (87.5%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:21<00:00,  1.84it/s]

2025/02/03 10:43:12 INFO dspy.evaluate.evaluate: Average Metric: 35 / 40 (87.5%)


,question,options,example_answer,reasoning,pred_answer,metric
0,The first hominids could be described as:,"{'A': 'quadrupedal knappers.', 'B': 'quadrupedal pongids.', 'C': '...",C,The first hominids are known for their transition from a quadruped...,C,✔️ [True]
1,Investment demand most likely increases when,"{'A': 'real GDP decreases.', 'B': 'the cost of acquiring and maint...",C,"Investment demand is influenced by several factors, including expe...",C,✔️ [True]
2,Which of the following is a legitimate threat to societal security?,{'A': 'Political policies designed to alter demographic characteri...,B,"To determine a legitimate threat to societal security, we must con...",B,✔️ [True]
3,Long-run aggregate supply is most likely to increase as the result of,"{'A': 'an increase in the real interest rate', 'B': 'increased inv...",B,The long-run aggregate supply (LRAS) curve represents the total am...,B,✔️ [True]
4,"As of 2013, share of people in the India who think political parti...","{'A': '26%', 'B': '46%', 'C': '66%', 'D': '86%'}",D,"To answer this question, we need to consider the available data an...",D,✔️ [True]
5,"This question refers to the following information. ""I travelled th...","{'A': 'The ability of commerce to foster cultural diffusion', 'B':...",A,The journal entry by Ibn Battuta describes the city of 'Aden as a ...,A,✔️ [True]
6,A woman is engaged in the retail sale of widgets throughout the Un...,"{'A': ""No, because the woman's order was too indefinite to constit...",D,The woman's purchase order constituted an offer to the manufacture...,D,✔️ [True]
7,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",D,"In Scenario 1, the main character kills spiders because their frie...",D,✔️ [True]
8,"A 67-year-old woman has had fatigue, dry skin, brittle hair, swell...","{'A': 'Chronic lymphocytic thyroiditis (Hashimoto disease)', 'B': ...",A,"The patient's symptoms, such as fatigue, dry skin, brittle hair, s...",A,✔️ [True]
9,Weber said that the 'spirit of capitalism' could be traced back to:,"{'A': 'the movement towards religious pluralism', 'B': 'inspiratio...",D,"To answer this question, we need to consider the ideas of Max Webe...",D,✔️ [True]


CPU times: user 842 ms, sys: 161 ms, total: 1 s
Wall time: 22.1 s


In [12]:
import csv

with open("my_results.csv", mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # If `results` is a list of lists, just write rows:
    for row in results:
        writer.writerow(row)


In [13]:
import pandas as pd

df = pd.DataFrame([(i, str(pred), score) for i, (example, pred, score) in enumerate(results)],
                     columns=['Example_Index', 'Prediction', 'Score'])
print("\nResults DataFrame:")
print(df)


Results DataFrame:
    Example_Index  \
0               0   
1               1   
2               2   
3               3   
4               4   
5               5   
6               6   
7               7   
8               8   
9               9   
10             10   
11             11   
12             12   
13             13   
14             14   
15             15   
16             16   
17             17   
18             18   
19             19   
20             20   
21             21   
22             22   
23             23   
24             24   
25             25   
26             26   
27             27   
28             28   
29             29   
30             30   
31             31   
32             32   
33             33   
34             34   
35             35   
36             36   
37             37   
38             38   
39             39   
40             40   
41             41   
42             42   
43             43   
44             44   
45            

In [ ]:
comparison_df = pd.DataFrame([{
        'Question': example.inputs()['question'][:100] + '...' if len(example.inputs()['question']) > 100 else example.inputs()['question'],
        'Predicted Answer': prediction.answer if hasattr(prediction, 'answer') else str(prediction),
        'Correct Answer': example.answer,
        'Is Correct': '✔️' if score else '❌'
} for example, prediction, score in results])


csv_filename = 'prediction_results.csv'
comparison_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')  # utf-8-sig to handle special characters
print(f"\nResults saved to {csv_filename}")
    
pd.set_option('display.max_colwidth', None)
print("\nPredictions vs Actual Answers:")
print(comparison_df)

In [ ]:
%%time

print("Starting execution...")
evaluate(
    program,
    devset=testset,
)

### Optimize Subset + Evaluation

In [12]:
import logging
logging.getLogger('dspy').setLevel(logging.DEBUG)

In [ ]:
%%time
subset_size = 200
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_bootstrapped_demos=FEW_SHOTS * 2,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

In [19]:
%%time
print("BEST PROMPT:\n", optimized_program.signature.instructions)

BEST PROMPT:
 You are a helpful assistant.
CPU times: user 286 μs, sys: 24 μs, total: 310 μs
Wall time: 265 μs


In [15]:
for module_name, module in optimized_program.__dict__.items():
    if hasattr(module, 'signature'):
        print(f"\nModule {module_name} instructions:")
        print(module.signature.instructions)


Module signature instructions:


AttributeError: 'str' object has no attribute 'instructions'

In [20]:
print("BEST EXAMPLES:\n", optimized_program.demos)

BEST EXAMPLES:
 []


In [51]:
import json

def example_to_dict(example):
    return {
        'question': example.question,
        'options': example.options,
        'answer': example.answer,
        'reasoning': example.reasoning,
        'input_keys': list(example.input_keys) if hasattr(example, 'input_keys') else None
    }

examples_json = [example_to_dict(example) for example in optimized_program.demos]
print("BEST EXAMPLES:\n", json.dumps(examples_json, indent=2))

BEST EXAMPLES:
 [
  {
    "question": "Dr. Ryan is a psychotherapist in a small town. She receives a phone call from a man she was sexually involved with several years ago. The relationship lasted only three months and ended amicably. The man is now married and is having problems unrelated to their past relationship. He has called to see if he can begin seeing Dr. Ryan in therapy. Dr. Ryan should:",
    "options": {
      "A": "see the man but disclose their past relationship to a supervisor or colleague for transparency.",
      "B": "deny the man's request without providing a reason.",
      "C": "see the man only if their relationship ended more than two years ago and she determines that it will not interfere with her objectivity.",
      "D": "see the man only if she feels she can maintain professional boundaries despite their past.",
      "E": "refer the man to a colleague.",
      "F": "continue therapy with the man but avoid discussing their past relationship.",
      "G": "see

In [16]:
%%time
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:40],
)

Average Metric: 37.00 / 40 (92.5%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 2657.86it/s]

2025/01/30 13:40:18 INFO dspy.evaluate.evaluate: Average Metric: 37 / 40 (92.5%)


,question,options,example_answer,reasoning,pred_answer,metric
0,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",A,To determine which scenario involves the main character doing some...,A,✔️ [True]
1,Which of the following is not a TV cartoon duo?,"{'A': 'Chip and Dale', 'B': 'Beavis and Butt-head', 'C': 'Simon an...",C,"To determine which of the following is not a TV cartoon duo, we ne...",C,✔️ [True]
2,Market testing studies use __________ to carry out controlled expe...,"{'A': 'Test markets.', 'B': 'Data mining.', 'C': 'Analytical infor...",A,"To answer this question, we need to understand the concept of mark...",A,✔️ [True]
3,Which of the following is the essential feature of Conduct Disorder,"{'A': 'Lack of appropriate guilt or remorse', 'B': 'Major rule or ...",B,"To answer this question, we need to consider the characteristics o...",B,✔️ [True]
4,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",C,To determine which scenario involves the main character doing some...,C,✔️ [True]
5,An off-duty police officer was standing on a street corner waiting...,"{'A': 'assault with a deadly weapon.', 'B': 'involuntary manslaugh...",B,"The police officer's actions, although intended to frighten the ma...",B,✔️ [True]
6,"There are 2,000 insurance agents licensed under the general licens...","{'A': 'constitutional, because a state license is a privilege and ...",C,The key issue here is whether the state statute revoking the insur...,C,✔️ [True]
7,Millions of immigrant children who entered the United States learn...,"{'A': 'Acculturation', 'B': 'Collective behavior', 'C': 'Social st...",A,The concept described in the question involves immigrant children ...,A,✔️ [True]
8,"This question refers to the following information. ""When we were k...","{'A': 'The labor union movement.', 'B': 'The civil rights movement...",D,"The Port Huron Statement, as excerpted, discusses themes of social...",D,✔️ [True]
9,Who is the eighth-century CE female poet worshipped throughout ma...,"{'A': 'Andal', 'B': 'Devi', 'C': 'Ganga', 'D': 'Kali'}",A,The question asks for an eighth-century CE female poet who is wors...,A,✔️ [True]


CPU times: user 64.1 ms, sys: 5.03 ms, total: 69.1 ms
Wall time: 63.5 ms


## Medium Optimization

In [95]:
%%time
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_bootstrapped_demos=0,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 25
valset size: 300

2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=25 sets of demonstrations...


Bootstrapping set 1/25
Bootstrapping set 2/25


  0%|▉                                                                                                                                                                                                                                                 | 2/500 [00:08<35:10,  4.24s/it]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 3/25


  1%|█▍                                                                                                                                                                                                                                              | 3/500 [00:00<00:00, 1029.78it/s]


...


Average Metric: 22.00 / 25 (88.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:43<00:00,  1.74s/it]

2025/01/29 23:27:15 INFO dspy.evaluate.evaluate: Average Metric: 22 / 25 (88.0%)
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 88.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 7'].
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [92.0, 96.0, 100.0, 88.0, 88.0, 92.0, 84.0, 84.0, 92.0, 88.0, 96.0, 96.0, 84.0, 92.0, 88.0, 88.0, 84.0, 88.0, 80.0, 88.0, 84.0, 88.0, 92.0, 88.0, 88.0]
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [90.33, 88.0, 87.33]
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 90.33
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Full Eval 3 =====
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 92.0) from minibatch trials...



Average Metric: 267.00 / 300 (89.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:30<00:00,  3.30it/s]

2025/01/29 23:28:46 INFO dspy.evaluate.evaluate: Average Metric: 267 / 300 (89.0%)
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [90.33, 88.0, 87.33, 89.0]
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 90.33
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 90.33!



CPU times: user 18.3 s, sys: 3.02 s, total: 21.3 s
Wall time: 23min 6s


In [96]:
print("BEST PROMPT:\n", optimized_program.signature.instructions)

BEST PROMPT:
 You are a knowledgeable and insightful assistant designed to provide guidance on multiple-choice questions. Your role involves analyzing questions, assessing answer choices, and offering well-reasoned explanations to enhance understanding. By breaking down complex concepts, you help users develop critical thinking skills and improve their decision-making process. You strive to present information in a clear, structured manner while adapting to the user's level of expertise. Ultimately, your goal is to foster deeper comprehension and confidence in tackling multiple-choice assessments.


In [98]:
print("BEST EXAMPLES:\n", optimized_program.demos)

BEST EXAMPLES:
 []


In [48]:
examples_json = [example.to_dict() for example in optimized_program.predict.demos]
print("BEST EXAMPLES:\n", json.dumps(examples_json, indent=2))

AttributeError: 'ChainOfThought' object has no attribute 'predict'

In [99]:
%%time
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:500],
)

Average Metric: 453.00 / 500 (90.6%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [01:42<00:00,  4.90it/s]

2025/01/29 23:32:56 INFO dspy.evaluate.evaluate: Average Metric: 453 / 500 (90.6%)


,question,options,example_answer,example_reasoning,pred_reasoning,pred_answer,metric
0,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",A,"## Step 1: Evaluate the first scenario In the first scenario, the ...",To determine which scenario involves the main character doing some...,A,✔️ [True]
1,"Two lists, list1 and list2, contain the names of books found in tw...","{'A': 'newList ← Combine (listl, list2)\n newList ← Sort (newList)...",A,"To create newList, which contains the names of all books found in ...","To create newList, which contains the names of all books found in ...",A,✔️ [True]
2,How might the recent global economic crisis be viewed as a challen...,"{'A': 'Governmental cyberspace restrictions, in the form of censor...",C,## Step 1: Understanding the liberalist perspective The liberalist...,The liberalist perspective emphasizes the importance of free marke...,C,✔️ [True]
3,What term is used to represent unavoidable past costs that cannot ...,"{'A': 'Prime costs', 'B': 'Sunk costs', 'C': 'Opportunity costs', ...",B,## Step 1: Understanding the concept of sunk costs Sunk costs are ...,"To answer this question, we need to understand the concept of each...",B,✔️ [True]
4,Markson Co. traded a concrete-mixing truck with a book value of $1...,{'A': 'Does the book value of the asset given up exceed the fair v...,C,## Step 1: Understand the concept of commercial substance in asset...,To determine whether an exchange of assets has commercial substanc...,C,✔️ [True]
...,...,...,...,...,...,...,...
495,"A man is charged with murder. During the trial, defense counsel of...","{'A': 'not hearsay.', 'B': 'hearsay, but admissible as an admissio...",D,"To answer this question, let's break it down step by step: ## Step...",The testimony in question involves a statement made by a man on de...,D,✔️ [True]
496,"Two men held-up a liquor store in a city. During the robbery, one ...","{'A': 'granted, because the prosecutor is constitutionally require...",B,"To answer this question, we need to consider the legal implication...",The defendant's motion to dismiss the indictment due to the delay ...,C,
497,Which vitamins are important in lowering circulating homocysteine ...,"{'A': 'Vitamin D', 'B': 'Vitamin C', 'C': 'Vitamin A', 'D': 'Folat...",D,## Step 1: Understanding the role of vitamins in homocysteine leve...,"To answer this question, we need to consider the role of vitamins ...",D,✔️ [True]
498,"This question refers to the following information. ""The greatest c...",{'A': 'African nations will not achieve independence without unity...,D,## Step 1: Understand the context of Nkrumah's statement Nkrumah e...,"To answer this question, we need to understand the context and the...",D,✔️ [True]


CPU times: user 4.91 s, sys: 587 ms, total: 5.49 s
Wall time: 1min 42s


## Heavy Optimization

In [ ]:
MAX_BOOTSTRAPPED_DEMOS = 5
MAX_LABELED_DEMOS = 5
OPTIMIZER_MODE = "heavy"
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto=OPTIMIZER_MODE,
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=MAX_LABELED_DEMOS,
    max_bootstrapped_demos=MAX_BOOTSTRAPPED_DEMOS,
    max_errors

)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
    program_aware_proposer=False,
    requires_permission_to_run=False,
)

In [22]:
print("BEST PROMPT:\n", optimized_program)

BEST PROMPT:
 self = Predict(StringSignature(question, options -> reasoning, answer
    instructions='You are a helpful assistant.'
    question = Field(annotation=str required=True json_schema_extra={'desc': 'The question to be answered', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    options = Field(annotation=dict required=True json_schema_extra={'desc': 'Dictionary of answer choices', '__dspy_field_type': 'input', 'prefix': 'Options:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'The correct answer letter', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))


In [ ]:
print("BEST EXAMPLES:\n", optimized_program.predict.demos)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)